# IUM projekt - etap 2
### Jakub Grzechociński  
### Piotr Sawicki


## Treść zadania
> Są osoby, które wchodzą na naszą stronę i nie mogą się zdecydować, którym produktom przyjrzeć się nieco lepiej. Może dało by się im coś polecić?

## Streszczenie etapu pierwszego
Do stworzenia jest system oferujący usługę rekomendacji produktów dla klientów sklepu internetowego. System opiera swoje działanie na danych przejrzeń i zakupów produktów przez użytkowników sklepu oraz na danych samych produktów.

System oferuje rekomendacje na podstawie dwóch oddzielnych modeli:
- `popularnościowy` bazujący na względnej popularności produktów i kategorii oraz preferencji użytkownika
- `collaborative filtering` odtwarzający podobieństwa między klientami

W ramach implementacji systemu należy utworzyć interfejs dla modeli rekomendacji w postaci mikroserwisu. W wyniku jego działania aplikacje klienckie mogą uzyskiwać rekomendacje produktów na podstawie id użytkownika. System pozwala na działanie z wykorzystaniem jednego z dwóch modeli lub w trybie testu A/B dla porównania modeli.

W etapie pierwszym wykonana została analiza danych, w ramach której zidentyfikowane zostały problemy oraz oszacowano, że system jest w stanie skutecznie rekomendować na podstawie danych.

## Zaimplementowane modele

### Model popularnościowy
Model opiera swoje działanie na względnej popularności przedmiotów oraz ich kategorii. Zliczana jest ilość wyświetlneń oraz zakupów każdego z produktów, przy czym zakup ma mnożnik X9 (jest to jeden z hiperparametrów modelu). Produkty są sortowane zgodnie z tak oszacowaną popularnością wewnątrz swoich karegorii. 

## Pokaz działania 

Serwis został umieszczony na maszynie w Oracle Cloud pod publicznym adresem ip `130.61.188.211`. Można więc przetestować jego działanie z dowolnego miejsca. Serwis odpowiada na odpowiednio sformułowane żądanie HTTP.   

Np. Aby uzyskać otrzymać odpowiedź zawierającą zestaw dziesięciu rekomendacji dla klienta o id `120` wystarczy wysłać zapytanie HTTP GET na adres: 
``` 
 130.61.188.211/recommendations?user_id=121&num=10 
``` 
W linii poleceń można szybko wykonać takie zapytanie przy pomocy narzędzia `curl`: 
``` 
curl -i "130.61.188.211/recommendations?user_id=121&num=10" 
``` 
Odpowiedź serwera: 
``` 
HTTP/1.1 200 OK 
Content-Type: application/json 
Date: Sun, 28 Mar 2021 16:30:32 GMT 
Content-Length: 110 
  
{"type":"user","id":121,"attributes":{"recommendations":[1077,1076,1080,1095,1004,1005,1006,1007,1008,1009]}} 
``` 

Serwer chodzący pod publicznym IP pracuje w trybie testu A/B. Przyporządkowanie użytkowników do modelu zapisywane jest w bazie danych SQLite. Po stronie serwera możemy sprawdzić, który system wygenerował ten zestaw rekomendacji: 
``` 
sqlite> SELECT * FROM user_to_model WHERE user_id == 121; 
user_id|model_id 
121|2 
``` 
Następnie szybki wgląd w przypisanie `model_id` do zrozumiałej nazwy: 
``` 
sqlite> SELECT * FROM models; 
model_id|name|user_count 
1|Popularity|4 
2|Collaborative|4 
``` 

Dla zapytania dla użytkowinika o `user_id` 121 otrzymaliśmy więc rekomendacje od systemu `Collaborative`, czyli modelu CF. Na podstawie tak zbieranych danych oraz dalszych danych zbieranych przez sklep internetowy można oszacować skuteczność obu modeli. 

Warto by było teraz sprawdzić jakie produkty zostały zarekomendowane. Są to produkty o `product_id`: 
```
1077,1076,1080,1095,1004,1005,1006,1007,1008,1009 
``` 

In [1]:
import datetime
import numpy as np
import pandas as pd

data_dir = 'data/'
data_raw_dir = 'data_raw/'
products_file = 'products.jsonl'
sessions_file = 'sessions.jsonl'
products_filepath = data_raw_dir + products_file
sessions_filepath = data_dir + sessions_file
products_data = pd.read_json(products_filepath, convert_dates=False, lines=True)
products_data = products_data.drop(columns=['price'])
sessions_data = pd.read_json(sessions_filepath, convert_dates=False, lines=True)

In [6]:
product_list = [1283,1001,1318,1277,1278,1276,1281,1035,1234,1067]
# get only the products from list
recommended_products = products_data[products_data.product_id.isin(product_list)]
# index them in the order from the list
recommended_products.reindex(recommended_products.product_id.map({x: i for i, x in enumerate(product_list)}).sort_values().index)

,product_id,product_name,category_path
282,1283,Okulary 3D PHILIPS PTA436/00,Sprzęt RTV;Video;Telewizory i akcesoria;Okular...
0,1001,Telefon Siemens Gigaset DA310,Telefony i akcesoria;Telefony stacjonarne
317,1318,Plantronics Savi W710,Sprzęt RTV;Audio;Słuchawki
276,1277,Apple iPad mini 64GB 4G,Komputery;Tablety i akcesoria;Tablety
277,1278,Intenso Music Walker 8GB,Sprzęt RTV;Przenośne audio i video;Odtwarzacze...
275,1276,Apple iPad mini 64GB,Komputery;Tablety i akcesoria;Tablety
280,1281,Manta MM266,Sprzęt RTV;Przenośne audio i video;Odtwarzacze...
34,1035,Samsung Galaxy S III GT-i9300,Telefony i akcesoria;Telefony komórkowe
233,1234,Sony DVP-SR760,Sprzęt RTV;Video;Odtwarzacze DVD
66,1067,Nokia 3310,Telefony i akcesoria;Telefony komórkowe


Produkty najlepsze zdaniem modelu są wyżej, gorsze niżej. Model zdaje się polecać klientowi przede wszystkim skanery.

In [3]:
target_user_id = 302
# Odrzucenie danych nie związanych z użytkownikiem o id 302
sessions_data_for_user = sessions_data.loc[sessions_data['user_id']==target_user_id]
# Złączenie danych sesji i produktów
session_join_products = sessions_data.join(products_data.set_index('product_id'), on='product_id', how='left')
session_join_products = session_join_products[['session_id', 'user_id', 'event_type', 'category_path', 'product_name']]
session_join_products

,session_id,user_id,event_type,category_path,product_name
0,100002,102,VIEW_PRODUCT,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...,One For All SV 9143
1,100002,102,VIEW_PRODUCT,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...,Telmor DSP-860
2,100002,102,VIEW_PRODUCT,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...,Philips SDV6224
3,100002,102,VIEW_PRODUCT,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...,Evolveo SHARK
4,100003,102,VIEW_PRODUCT,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,Jabra Freeway
...,...,...,...,...,...
83035,110659,302,VIEW_PRODUCT,Gry i konsole;Gry na konsole;Gry Xbox 360,Dead Space 3 (Xbox 360)
83036,110659,302,VIEW_PRODUCT,Gry i konsole;Gry na konsole;Gry PlayStation3,GTA 5 (PS3)
83037,110659,302,VIEW_PRODUCT,Gry i konsole;Gry na konsole;Gry Xbox 360,GTA 4 (Xbox 360)
83038,110659,302,VIEW_PRODUCT,Gry i konsole;Gry na konsole;Gry Xbox 360,Mass Effect 2 (Xbox 360)


In [4]:
session_join_products_view = session_join_products[session_join_products['event_type']=='VIEW_PRODUCT']
session_join_products_view['product_name'].value_counts().head(60)

Okulary 3D PHILIPS PTA436/00                      675
Plantronics Savi W710                             654
Telefon Siemens Gigaset DA310                     623
Apple iPad mini 64GB                              537
Nokia 3310                                        533
Samsung Galaxy S III GT-i9300                     520
Apple iPad mini 64GB 4G                           511
Manta MM266                                       501
Manta DVD064                                      498
Intenso Music Walker 8GB                          472
Sony DVP-SR760                                    454
Plantronics Savi W740                             450
HTC HS-S200                                       441
Plantronics Voyager Legend                        436
Jabra Speak 410                                   417
Jabra Talk                                        417
Jabra Freeway                                     414
Gembird BTCC-002                                  407
Jabra Drive                 

In [5]:
session_join_products_view_okulary = session_join_products_view[session_join_products_view['product_name']=='Okulary 3D PHILIPS PTA436/00']
session_join_products_view_okulary['user_id'].value_counts()

162    13
138    12
127    11
256     9
167     9
       ..
151     1
253     1
150     1
263     1
301     1
Name: user_id, Length: 180, dtype: int64